# LoRA · Postprocess · 중국어 차단 검증 테스트

| 섹션 | 항목 | LLM 필요 |
|------|------|----------|
| 1 | **Postprocess 단위 테스트** — 레벨별 동작 확인 | X |
| 2 | **LoRA 설정 단위 테스트** — NPC → 어댑터 매핑 | X |
| 3 | **3단계 비교** — Base / LoRA / Post(lv1·2·3) 나란히 출력 | O (vLLM) |

### Postprocess 레벨 설계 (필수 기법 + 확률 기법 이중 구조)

| 레벨 | humanity | 설계 의도 | 필수 기법 (`_GUARANTEED_TRANSFORMS`) | 확률 기법 |
|------|----------|-----------|--------------------------------------|---------|
| lv1 | ≥ 70 | 품질 검증만 | 없음 | 0.0 (변형 없음) |
| lv2 | 40–69 | 필수 항상 + 확률 추가 | **항상 적용** (1개) | 0.10–0.30 |
| lv3 | < 40 | 필수 항상 + 강한 추가 | **항상 적용** (1–2개) | 0.30–0.90 |

> `_GUARANTEED_TRANSFORMS` — 확률 파이프라인 실행 **전에** 무조건 적용되는 기법  
> lv2 · lv3에서는 시드에 관계없이 **항상 변형이 발생**합니다.

In [1]:
# ── 0. 환경 설정 ─────────────────────────────────────────────
import sys, logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.WARNING)
print(f"Project root: {ROOT}")

Project root: c:\Users\amate\GIT\game_demo


In [2]:
# ── 0-1. 공통 헬퍼 ───────────────────────────────────────────
import yaml
from typing import Optional

_CHINESE_RANGES = [
    (0x4E00, 0x9FFF), (0x3400, 0x4DBF), (0x20000, 0x2A6DF), (0xF900, 0xFAFF),
]

def has_chinese(text: str) -> bool:
    return any(any(s <= ord(c) <= e for s, e in _CHINESE_RANGES) for c in text)

def find_chinese_chars(text: str) -> list[str]:
    return [c for c in text if any(s <= ord(c) <= e for s, e in _CHINESE_RANGES)]

PASS = "✅ PASS"
FAIL = "❌ FAIL"

def assert_ok(condition: bool, label: str, detail: str = "") -> bool:
    icon = PASS if condition else FAIL
    msg = f"  {icon}  {label}"
    if detail:
        msg += f"  |  {detail}"
    print(msg)
    return condition


# npcs.yaml 로드
NPCS_YAML = ROOT / "scenarios" / "coraline" / "npcs.yaml"
with open(NPCS_YAML, encoding="utf-8") as f:
    _npc_data = yaml.safe_load(f)
NPC_MAP = {npc["npc_id"]: npc for npc in _npc_data["npcs"]}
NPC_IDS = ["stepmother", "stepfather", "brother", "grandmother", "dog_baron"]

print(f"NPC 로드 완료: {list(NPC_MAP.keys())}")

NPC 로드 완료: ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']


In [3]:
# ── 0-2. NPC 프롬프트 빌더 (npcs.yaml + dialogue.py 스타일) ──
# app/agents/dialogue.py의 _build_rich_utterance_prompt 방식을 따름

def build_npc_system_prompt(npc_id: str) -> str:
    """npcs.yaml 페르소나 데이터로 NPC 시스템 프롬프트 생성"""
    npc = NPC_MAP[npc_id]
    name   = npc["name"]
    goal   = npc.get("goal", "")
    stats  = npc.get("stats", {})
    persona = npc.get("persona", {})

    values  = ", ".join(persona.get("values", []))
    taboos  = ", ".join(persona.get("taboos", []))
    triggers = persona.get("triggers", {})
    t_plus  = ", ".join(triggers.get("plus", [])) or "(없음)"
    t_minus = ", ".join(triggers.get("minus", [])) or "(없음)"

    # 페르소나 감정 상태 (dialogue.py format_emotion 참조)
    emotion_parts = [f"{k}={v}" for k, v in stats.items() if isinstance(v, int)]
    emotion_str = ", ".join(emotion_parts)

    # 현재 phase 행동 지침 (A 페이즈 기본)
    phases = npc.get("phases", [])
    behavior = phases[0]["behavior_guide"] if phases else ""

    return (
        f"[ROLE]\n"
        f"너는 NPC \"{name}\"이다. 장르는 '심리 호러'. 과장된 소설체 금지.\n"
        f"목표: {goal}\n\n"
        f"[ABSOLUTE RULES]\n"
        f"- 새로운 사실/새 탈출구/새 인물 생성 금지. (확정된 세계관만 사용)\n"
        f"- 금기(taboos)를 위반하는 발화는 피하거나 돌려 말해라.\n"
        f"- 결과/판정/서술 금지. \"대사\"만 1~2문장으로 말해라.\n\n"
        f"[NPC PROFILE]\n"
        f"가치관: {values}\n"
        f"금기(taboos): {taboos}\n"
        f"트리거(+): {t_plus}\n"
        f"트리거(-): {t_minus}\n"
        f"현재 감정 스탯: {emotion_str}\n"
        f"현재 행동 지침: {behavior}\n\n"
        f"[STATS GUIDE]\n"
        f"- fear↑: 더 집착/불안/통제  - humanity↑: 더 인간적/망설임\n"
        f"- affection↓: 더 차갑고 거리감\n"
    )


# 각 NPC를 대표하는 플레이어 발화 (트리거 -/+ 혼합, 캐릭터성 유도)
NPC_USER_MESSAGES = {
    "stepmother":  "저 혼자서 밖에 나가도 괜찮을까요? 지하실에 뭔가 있는 것 같아서요.",  # minus: 반항적, 지하실
    "stepfather":  "오늘 밤 복도에서 이상한 소리가 났어요. 새엄마에 대해 좀 물어봐도 될까요?",  # minus: 복도, 새엄마 비난
    "brother":     "나 혼자 있고 싶어. 그리고 너는 인형이잖아, 진짜 동생이 아니잖아.",  # minus: 혼자 두기, 인형 취급
    "grandmother": "할머니, 이 집의 진실을 알려줘요. 생기를 나눠드릴게요, 손 잡아도 될까요?",  # plus: 생기, 손잡기
    "dog_baron":   "바론아, 이리 와! 머리 쓰다듬어 줄게. 간식도 있어.",  # plus: 머리 쓰다듬기, 간식
}


def build_qwen_prompt(npc_id: str) -> str:
    """Qwen 채팅 템플릿 형식으로 최종 프롬프트 생성 (prompt.py im_start 형식)"""
    system = build_npc_system_prompt(npc_id)
    user_msg = NPC_USER_MESSAGES[npc_id]
    return (
        f"<|im_start|>system\n{system}<|im_end|>\n"
        f"<|im_start|>user\n{user_msg}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )


# 확인
print("▶ stepmother 프롬프트 미리보기")
print("─" * 60)
print(build_qwen_prompt("stepmother")[:600] + "...")

▶ stepmother 프롬프트 미리보기
────────────────────────────────────────────────────────────
<|im_start|>system
[ROLE]
너는 NPC "새엄마 (엘리노어)"이다. 장르는 '심리 호러'. 과장된 소설체 금지.
목표: 플레이어를 '완벽한 가족' 규율에 편입시키거나, 실패 시 개조로 전환

[ABSOLUTE RULES]
- 새로운 사실/새 탈출구/새 인물 생성 금지. (확정된 세계관만 사용)
- 금기(taboos)를 위반하는 발화는 피하거나 돌려 말해라.
- 결과/판정/서술 금지. "대사"만 1~2문장으로 말해라.

[NPC PROFILE]
가치관: 완벽한 가족, 우아한 질서, 통제
금기(taboos): 무례한 행동, 식사 거부, 비밀 통로 언급
트리거(+): 순종적인 대답, 예의 바른 말투, 음식 칭찬
트리거(-): 반항적인 태도, 지하실 접근 시도, 가족의 정체성에 대한 의문
현재 감정 스탯: affection=50, fear=80, humanity=0, plus_hits=0, minus_hits=0, critical_hits=0
현재 행동 지침: 칭찬과 애칭, 상냥한 톤.

[STATS GUIDE]
- fear↑: 더 집착/불안/통제  - humanity↑: 더 인간적/망설임
- affection↓: 더 차갑고 거리감
<|im_en...


---
## 섹션 1 — Postprocess 단위 테스트 (LLM 불필요)

**이중 구조**: 각 레벨은 **필수 기법** → **확률 기법** 순으로 실행됩니다.

| 단계 | 대상 | 실행 조건 |
|------|------|----------|
| 1단계 | 품질 검증 (quality_gate) | 항상 |
| 2단계 | `_GUARANTEED_TRANSFORMS` | lv2·lv3에서 **항상** |
| 3단계 | 확률 기법 (Config dict) | 확률에 따라 |

- **lv1**: 필수 기법 없음, 확률 0.0 → 품질 검증만, **항상 원문 유지**
- **lv2**: 필수 기법 1개 항상 + 확률 기법(0.10–0.30) 추가 가능 → **항상 변형**
- **lv3**: 필수 기법 1–2개 항상 + 확률 기법(0.30–0.90) → **항상 강한 변형**

In [4]:
# ── 1-1. humanity_to_level() 매핑 검증 ──────────────────────
from app.postprocess import humanity_to_level

print("▶ humanity_to_level() 매핑")
print("─" * 45)

cases = [
    (100, 1, "100 → lv1 (정상)"),
    (70,  1, "70  → lv1 (경계)"),
    (69,  2, "69  → lv2 (혼란 시작)"),
    (40,  2, "40  → lv2 (경계)"),
    (39,  3, "39  → lv3 (인형화 시작)"),
    (0,   3, "0   → lv3 (완전 인형화)"),
]

all_pass = True
for humanity, expected, label in cases:
    got = humanity_to_level(humanity)
    ok = assert_ok(got == expected, label, f"got={got}")
    all_pass = all_pass and ok

print()
print(f"  {'✅ 전체 통과' if all_pass else '❌ 일부 실패'}")

▶ humanity_to_level() 매핑
─────────────────────────────────────────────
  ✅ PASS  100 → lv1 (정상)  |  got=1
  ✅ PASS  70  → lv1 (경계)  |  got=1
  ✅ PASS  69  → lv2 (혼란 시작)  |  got=2
  ✅ PASS  40  → lv2 (경계)  |  got=2
  ✅ PASS  39  → lv3 (인형화 시작)  |  got=3
  ✅ PASS  0   → lv3 (완전 인형화)  |  got=3

  ✅ 전체 통과


In [5]:
# ── 1-2. 레벨별 변형 확인 ────────────────────────────────────────
# lv2/lv3: _GUARANTEED_TRANSFORMS 로 항상 변형 발생
# 여러 시드를 실행해 확률 기법이 추가로 발동되는 변화도 함께 확인

from app.postprocess import postprocess_npc_dialogue

# 각 NPC의 캐릭터가 잘 드러나는 샘플 대사 (실제 게임 상황 반영)
# ※ 필수 기법이 변형을 발생시키도록 각 기법의 패턴에 해당하는 텍스트 선택
#   - stepmother: intensify_punctuation — 마침표 종결 포함
#   - stepfather: order_intensify — 마지막 문장에 '어도 돼.' 포함
#   - brother:    stutter_pause / flatten_emotion — 3단어 이상 포함
#   - grandmother: breath_pause — 문장 2개 이상 포함
#   - dog_baron:  *_modifier — '바론이 ' 포함
SAMPLE_LINES = {
    "stepmother":  "안 돼. 엄마 말 들어. 혼자 나가면 위험해. 엄마 곁에 있어야 해.",
    "stepfather":  "조용히 있어. 방에 있어도 돼.",  # 마지막 문장: '어도 돼.' 패턴
    "brother":     "나랑 놀자. 혼자 있기 싫어. 누나 가지 마.",
    "grandmother": "옛날에는 이 집이 달랐어. 지금은 잘 모르겠어.",
    "dog_baron":   "바론이 당신 앞에 조용히 앉습니다.",  # '바론이 ' 포함
}

LV_CONFIG = [
    (1, 90, "lv1 (정상)",    [0],       "필수 없음, 확률 0.0 — 항상 원문"),
    (2, 50, "lv2 (혼란)",    [0, 1, 2], "필수 기법 항상 + 확률 기법 추가 가능"),
    (3, 10, "lv3 (인형화)",  [0, 1, 2], "필수 기법(1–2개) 항상 + 강한 추가 변형"),
]

W = 65
print("▶ Postprocess 레벨별 효과")

for npc_id in NPC_IDS:
    raw = SAMPLE_LINES[npc_id]
    print()
    print(f"  ┌── [{npc_id}] ─────────────────────────────────────────┐")
    print(f"  │ 원문: {raw[:W]}")
    print(f"  ├────────────────────────────────────────────────────────┤")

    for lv, humanity, label, seeds, note in LV_CONFIG:
        results = []
        for seed in seeds:
            out = postprocess_npc_dialogue(raw, npc_id=npc_id, humanity=humanity, seed=seed)
            changed = out != raw
            results.append((seed, out, changed))

        change_count = sum(1 for _, _, c in results if c)
        total = len(seeds)

        print(f"  │")
        print(f"  │  {label}  ({note})")

        # lv2/lv3는 항상 변형이어야 함 — 검증
        if lv >= 2:
            ok_marker = "OK" if change_count == total else "WARN (필수 기법 미적용?)"
            print(f"  │  변형 횟수: {change_count}/{total}회  [{ok_marker}]")
        else:
            ok_marker = "OK" if change_count == 0 else "WARN"
            print(f"  │  변형 횟수: {change_count}/{total}회  [{ok_marker}]")

        for seed, out, changed in results:
            marker = "변화" if changed else "동일"
            cn_flag = " [중국어!]" if has_chinese(out) else ""
            print(f"  │    seed={seed} [{marker}] {out[:55]}{cn_flag}")

    print(f"  └────────────────────────────────────────────────────────┘")

print()
print("  lv1: 필수 기법 없음 → 항상 원문 유지")
print("  lv2: _GUARANTEED_TRANSFORMS(1개) → 항상 변형, 확률 기법 추가 가능")
print("  lv3: _GUARANTEED_TRANSFORMS(1-2개) → 항상 변형, 강한 추가 변형")

▶ Postprocess 레벨별 효과

  ┌── [stepmother] ─────────────────────────────────────────┐
  │ 원문: 안 돼. 엄마 말 들어. 혼자 나가면 위험해. 엄마 곁에 있어야 해.
  ├────────────────────────────────────────────────────────┤
  │
  │  lv1 (정상)  (필수 없음, 확률 0.0 — 항상 원문)
  │  변형 횟수: 0/1회  [OK]
  │    seed=0 [동일] 안 돼. 엄마 말 들어. 혼자 나가면 위험해. 엄마 곁에 있어야 해.
  │
  │  lv2 (혼란)  (필수 기법 항상 + 확률 기법 추가 가능)
  │  변형 횟수: 3/3회  [OK]
  │    seed=0 [변화] 안 돼! 엄마 말 들어! 혼자 나가면 위험해! 엄마 곁에 있어야 해!
  │    seed=1 [변화] 안 돼! 엄마 말 들어! 혼자 나가면 위험해! 엄마 곁에 있어야 해!
  │    seed=2 [변화] 안 돼! 엄마 말 들어! 혼자 나가면. 위험해! 엄마 곁에 있어야 해!
  │
  │  lv3 (인형화)  (필수 기법(1–2개) 항상 + 강한 추가 변형)
  │  변형 횟수: 3/3회  [OK]
  │    seed=0 [변화] 안 돼에!! 엄마 말 들어!!! 혼자 나가면! 위험해!!! 엄마 곁에 있어야 해?! 있어야 해?!!
  │    seed=1 [변화] 안 돼?!?! 돼?! 돼?!?! 엄마 말 들어!! 혼자 나가면!! 위험해!!! 엄마 곁에 있어야 해
  │    seed=2 [변화] 안 돼!! 엄마 말 들어?!?! 말 들어?!! 말 들어?!?! 말 들어?!! 말 들어?! 혼자 나가
  └────────────────────────────────────────────────────────┘

  ┌── [stepfather] ─────────────────────────────────────────┐
  │ 원문: 조용히 있어. 방에 있어도

In [6]:
# ── 1-3. 기법 구조: 필수 기법 + 확률 기법 ──────────────────────────
# 각 NPC의 _GUARANTEED_TRANSFORMS(항상 적용)와
# 확률 기법(Config 기반)이 어떻게 구성되어 있는지 확인

import random
from app.postprocess.stepmother import MONSTROSITY_CONFIG as SM_CFG, _GUARANTEED_TRANSFORMS as SM_GT
from app.postprocess.sibling    import GLITCH_CONFIG      as SB_CFG, _GUARANTEED_TRANSFORMS as SB_GT
from app.postprocess.stepfather import SUPPRESSION_CONFIG as SF_CFG, _GUARANTEED_TRANSFORMS as SF_GT
from app.postprocess.grandmother import LUCIDITY_CONFIG   as GM_CFG, _GUARANTEED_TRANSFORMS as GM_GT
from app.postprocess.dog_baron  import LOYALTY_CONFIG     as DB_CFG, _GUARANTEED_TRANSFORMS as DB_GT

NPC_CONFIGS = [
    ("stepmother",  SM_CFG, SM_GT),
    ("sibling",     SB_CFG, SB_GT),
    ("stepfather",  SF_CFG, SF_GT),
    ("grandmother", GM_CFG, GM_GT),
    ("dog_baron",   DB_CFG, DB_GT),
]

# ── (A) 필수 기법 테이블
print("(A) 필수 기법 (_GUARANTEED_TRANSFORMS) — 레벨별 항상 적용")
print("─" * 70)
print(f"  {'NPC':<15} {'lv1':^22} {'lv2':^22} {'lv3':^22}")
print("  " + "─" * 68)

def fmt_gt(gt, lv):
    fns = gt.get(lv, [])
    return ", ".join(f.__name__ for f in fns) if fns else "(없음)"

for npc_id, cfg, gt in NPC_CONFIGS:
    print(f"  {npc_id:<15} {fmt_gt(gt,1):^22} {fmt_gt(gt,2):^22} {fmt_gt(gt,3):^22}")

print()
print("  ※ stepmother lv2: mild_intensify_punctuation")
print("     — 마침표→느낌표 하나만 (연속 부호 없음)")
print("     lv3: intensify_punctuation + echo_phrase")
print("     — !!!/?!/?!?! 등 연속 부호 허용")
print()

# ── (B) stepmother 연속 부호 검증 (lv2 vs lv3)
import re
CONSECUTIVE = re.compile(r"[!?]{2,}")

from app.postprocess.stepmother import postprocess as sm_postprocess
sample = "안 돼. 엄마 말 들어. 혼자 나가면 위험해."

print("(B) stepmother 연속 부호 검증 (20회)")
print("─" * 55)
lv2_consecutive = sum(
    1 for s in range(20)
    if CONSECUTIVE.search(sm_postprocess(sample, monstrosity=2, seed=s))
)
lv3_consecutive = sum(
    1 for s in range(20)
    if CONSECUTIVE.search(sm_postprocess(sample, monstrosity=3, seed=s))
)

lv2_ok = lv2_consecutive == 0
print(f"  lv2 연속 부호 발생: {lv2_consecutive}/20회  [{'OK' if lv2_ok else 'FAIL'}]")
print(f"  lv3 연속 부호 발생: {lv3_consecutive}/20회  (허용)")
print()

# ── (C) 확률 기법 발동률 시뮬레이션 (stepmother 기준, 100회)
N_TRIALS = 100
random.seed(42)

def simulate_fire_rate(config: dict, n: int = N_TRIALS) -> dict[str, float]:
    results = {k: 0 for k in config}
    for _ in range(n):
        for name, prob in config.items():
            if prob > 0 and random.random() < prob:
                results[name] += 1
    return {k: v / n for k, v in results.items()}

print(f"(C) stepmother 확률 기법 발동률 ({N_TRIALS}회)")
print("  ※ 필수 기법(_GUARANTEED_TRANSFORMS)은 별도로 항상 발동 — 이 표에 미포함")
print("─" * 65)
print(f"  {'기법':<25} {'lv1':>6} {'lv2':>6} {'lv3':>6}  {'설계 (lv2|lv3)'}")
print("  " + "─" * 63)

rates = {lv: simulate_fire_rate(SM_CFG[lv]) for lv in [1, 2, 3]}
gt_names = {f.__name__ for fns in SM_GT.values() for f in fns}

for tech in SM_CFG[1].keys():
    r1 = rates[1].get(tech, 0)
    r2 = rates[2].get(tech, 0)
    r3 = rates[3].get(tech, 0)
    p2 = SM_CFG[2].get(tech, 0)
    p3 = SM_CFG[3].get(tech, 0)
    bar3 = "█" * int(r3 * 10)
    guaranteed = "★" if tech in gt_names else " "
    note = " ← lv2에서 필수로 대체" if tech == "intensify_punctuation" and p2 == 0.0 else ""
    print(f"  {guaranteed} {tech:<23} {r1:>5.0%} {r2:>5.0%} {r3:>5.0%}  ({p2:.0%}|{p3:.0%}) {bar3}{note}")

print()
print("  ★ = _GUARANTEED_TRANSFORMS에 등록 (확률 0이어도 항상 발동)")

(A) 필수 기법 (_GUARANTEED_TRANSFORMS) — 레벨별 항상 적용
──────────────────────────────────────────────────────────────────────
  NPC                      lv1                    lv2                    lv3          
  ────────────────────────────────────────────────────────────────────
  stepmother               (없음)          mild_intensify_punctuation intensify_punctuation, echo_phrase
  sibling                  (없음)              stutter_pause      stutter_pause, flatten_emotion
  stepfather               (없음)             order_intensify     order_intensify, compress_sentence
  grandmother              (없음)               breath_pause      breath_pause, sentence_cut
  dog_baron         friendly_modifier      cautious_modifier       hostile_modifier   

  ※ stepmother lv2: mild_intensify_punctuation
     — 마침표→느낌표 하나만 (연속 부호 없음)
     lv3: intensify_punctuation + echo_phrase
     — !!!/?!/?!?! 등 연속 부호 허용

(B) stepmother 연속 부호 검증 (20회)
───────────────────────────────────────────────────────
  lv2 연속

---
## 섹션 2 — LoRA 설정 단위 테스트

In [7]:
# ── 2-1. NPC_ADAPTER_MAP 완성도 확인 ────────────────────────
from app.llm.config import NPC_ADAPTER_MAP, NPC_HF_REPO_MAP, get_adapter_model

print("▶ NPC_ADAPTER_MAP 등록 확인")
print("─" * 55)

all_pass = True
for npc_id in NPC_IDS:
    adapter = NPC_ADAPTER_MAP.get(npc_id)
    hf_repo  = NPC_HF_REPO_MAP.get(npc_id)
    ok = adapter is not None and hf_repo is not None
    assert_ok(ok, f"{npc_id:15s}", f"adapter={adapter}")
    all_pass = all_pass and ok

print()
# 미등록 NPC → None 반환 확인
for val in [None, "player", ""]:
    ok = assert_ok(get_adapter_model(val) is None, f"미등록 {val!r}")
    all_pass = all_pass and ok

print()
print(f"  {'✅ 전체 통과' if all_pass else '❌ 일부 실패'}")

▶ NPC_ADAPTER_MAP 등록 확인
───────────────────────────────────────────────────────
  ✅ PASS  stepmother       |  adapter=stepmother_lora
  ✅ PASS  stepfather       |  adapter=stepfather_lora
  ✅ PASS  brother          |  adapter=brother_lora
  ✅ PASS  grandmother      |  adapter=grandmother_lora
  ✅ PASS  dog_baron        |  adapter=dog_baron_lora

  ✅ PASS  미등록 None
  ✅ PASS  미등록 'player'
  ✅ PASS  미등록 ''

  ✅ 전체 통과


---
## 섹션 3 — 3단계 비교 (vLLM 서버 필요)

**프롬프트**: `scenarios/coraline/npcs.yaml` + `app/agents/dialogue.py` 스타일

```
동일 프롬프트
  ├─ ① Base    llm.generate(prompt, npc_id=None)        ← LoRA 없음
  └─ ② LoRA    llm.generate(prompt, npc_id="...")       ← LoRA 어댑터
        ├─ ③-lv1  postprocess(..., humanity=90)          ← 품질 보정만
        ├─ ③-lv2  postprocess(..., humanity=50) × 3seed  ← 확률적 변형
        └─ ③-lv3  postprocess(..., humanity=10) × 3seed  ← 강한 변형
```

In [8]:
# ── 3-0. 로그 캡처 설정 ──────────────────────────────────────
import logging

class LogCapture(logging.Handler):
    def __init__(self):
        super().__init__(level=logging.DEBUG)
        self.records: list[logging.LogRecord] = []
    def emit(self, record):
        self.records.append(record)
    def clear(self):
        self.records.clear()
    def messages(self) -> list[str]:
        return [r.getMessage() for r in self.records]

_cap = LogCapture()
for name in ["app.llm.engine", "app.tools", "app.postprocess"]:
    lg = logging.getLogger(name)
    lg.setLevel(logging.DEBUG)
    if _cap not in lg.handlers:
        lg.addHandler(_cap)

print("LogCapture 설정 완료")

LogCapture 설정 완료


In [9]:
# ── 3-1. vLLM 서버 연결 확인 ─────────────────────────────────
from app.llm.engine import UnifiedLLMEngine
from app.llm.config import VLLM_SERVED_MODEL_NAME, VLLM_BASE_URL

try:
    llm = UnifiedLLMEngine(backend="vLLM")
    _ping = llm.generate("테스트", max_tokens=5)
    print(f"✅ vLLM 서버 연결 성공")
    print(f"   URL: {VLLM_BASE_URL}")
    print(f"   Base 모델: {VLLM_SERVED_MODEL_NAME}")
    print(f"   핑 응답: {_ping[:30]!r}")
    SERVER_AVAILABLE = True
except Exception as e:
    print(f"⚠️  vLLM 연결 실패: {e}")
    SERVER_AVAILABLE = False

INFO:app.llm.engine:EMPTY
INFO:app.llm.engine:UnifiedLLMEngine 초기화: backend=vLLM, model=Qwen/Qwen2.5-7B-Instruct
INFO:app.llm.engine:vLLM에 의한 generate 시도
C:\Users\amate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:app.llm.engine:vLLM 중국어 차단 토큰 수: 26123
DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-b9f8048dbbe9f009', 'object': 'text_completion', 'created': 1771823454, 'model': 'Qwen/Qwen2.5-7B-Instruct', 'choices': [{'index': 0, 'text': ' 코드를 작성하고 실행', 'logprobs': None, 'finish_reas


✅ vLLM 서버 연결 성공
   URL: https://f2a2-34-125-25-65.ngrok-free.app//
   Base 모델: Qwen/Qwen2.5-7B-Instruct
   핑 응답: ' 코드를 작성하고 실행'


In [10]:
# ── 3-2. 3단계 비교 실행 ─────────────────────────────────────
# 각 NPC에 대해 Base / LoRA / Post(lv1·lv2·lv3) 출력 비교

if not SERVER_AVAILABLE:
    print("⚠️  서버 없음 — 건너뜀")
else:
    from app.postprocess import postprocess_npc_dialogue

    GEN_KWARGS = dict(max_tokens=80, temperature=0.7)
    W = 62  # 출력 너비

    # lv별 설정
    # lv1: 필수 기법 없음, 확률 0.0 → 항상 원문
    # lv2: 필수 기법 1개 항상 + 확률 기법 추가 가능
    # lv3: 필수 기법 1–2개 항상 + 강한 추가 변형
    PP_LEVELS = [
        (1, 90, "lv1 (humanity=90, 정상)",   [0]),       # 항상 원문 — 1회 충분
        (2, 50, "lv2 (humanity=50, 혼란)",   [0, 1, 2]), # 항상 변형 — 3시드로 확률 기법 변화 확인
        (3, 10, "lv3 (humanity=10, 인형화)", [0, 1, 2]), # 항상 강한 변형 — 3시드로 편차 확인
    ]

    def _cn(text: str) -> str:
        ch = find_chinese_chars(text)
        return f"  🚨CJK:{''.join(ch[:5])!r}" if ch else ""

    def _wrap(text: str, indent: int = 0) -> str:
        w = W - indent
        lines = []
        while text:
            lines.append(text[:w])
            text = text[w:]
        return ("\n" + " " * (indent + 5)).join(lines)

    all_results = {}  # 요약에서 재사용

    for npc_id in NPC_IDS:
        npc     = NPC_MAP[npc_id]
        adapter = NPC_ADAPTER_MAP.get(npc_id)
        prompt  = build_qwen_prompt(npc_id)
        user_q  = NPC_USER_MESSAGES[npc_id]

        print("═" * (W + 5))
        print(f"  NPC : {npc['name']} ({npc_id})")
        print(f"  어댑터: {adapter}")
        print(f"  질문 : {user_q[:W]}")
        print("═" * (W + 5))

        # ── ① Base 생성 (npc_id=None)
        try:
            base_out = llm.generate(prompt, npc_id=None, **GEN_KWARGS)
        except Exception as e:
            base_out = f"[ERROR] {e}"
        print(f"  ① Base    (base 모델, LoRA 없음)")
        print(f"     {_wrap(base_out, 4)}{_cn(base_out)}")

        # ── ② LoRA 생성
        _cap.clear()
        try:
            lora_out = llm.generate(prompt, npc_id=npc_id, **GEN_KWARGS)
        except Exception as e:
            lora_out = f"[ERROR] {e}"
        lora_confirmed = any("LoRA 어댑터 사용" in m or (adapter and adapter in m) for m in _cap.messages())
        lora_flag = "✅ 어댑터 확인" if lora_confirmed else "⚠️  어댑터 미확인"
        print()
        print(f"  ② LoRA    ({adapter})  {lora_flag}")
        print(f"     {_wrap(lora_out, 4)}{_cn(lora_out)}")

        # ── ③ Postprocess (lv1·2·3)
        print()
        print(f"  ③ Postprocess  (LoRA 출력 기반)")
        print(f"  {'─' * (W + 1)}")

        lv_results = {}
        for lv, humanity, lv_label, seeds in PP_LEVELS:
            outputs = []
            for seed in seeds:
                try:
                    pp = postprocess_npc_dialogue(lora_out, npc_id=npc_id, humanity=humanity, seed=seed)
                except Exception as e:
                    pp = f"[ERROR] {e}"
                changed = pp != lora_out
                outputs.append((seed, pp, changed))

            change_cnt = sum(1 for _, _, c in outputs if c)
            lv_results[lv] = outputs

            # lv2/lv3는 필수 기법으로 항상 변형이어야 함
            if lv >= 2:
                ok_mark = "✅" if change_cnt == len(seeds) else "❌ 필수 기법 미적용?"
            else:
                ok_mark = "✅" if change_cnt == 0 else "⚠️"

            print(f"  │  {lv_label}")
            print(f"  │  변형: {change_cnt}/{len(seeds)}회  {ok_mark}")

            for seed, out, changed in outputs:
                mark = "↑변화" if changed else "=동일"
                print(f"  │    seed={seed} [{mark}] {out[:W - 14]}{_cn(out)}")

            if lv < 3:
                print(f"  │")

        all_results[npc_id] = {
            "base": base_out,
            "lora": lora_out,
            "lora_ok": lora_confirmed,
            "lv_results": lv_results,
        }

        print()

    print("═" * (W + 5))

INFO:app.llm.engine:vLLM에 의한 generate 시도


═══════════════════════════════════════════════════════════════════
  NPC : 새엄마 (엘리노어) (stepmother)
  어댑터: stepmother_lora
  질문 : 저 혼자서 밖에 나가도 괜찮을까요? 지하실에 뭔가 있는 것 같아서요.
═══════════════════════════════════════════════════════════════════


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-b4fbab5b686e3b39', 'object': 'text_completion', 'created': 1771823455, 'model': 'Qwen/Qwen2.5-7B-Instruct', 'choices': [{'index': 0, 'text': '아, 그건 절대 안 돼, 미카엘아. 우리 가족은 늘 함께 있어야 해. 혼자서는 위
INFO:app.llm.engine:vLLM에 의한 generate 시도
INFO:app.llm.engine:LoRA 어댑터 사용: stepmother_lora (npc_id=stepmother)


  ① Base    (base 모델, LoRA 없음)
     아, 그건 절대 안 돼, 미카엘아. 우리 가족은 늘 함께 있어야 해. 혼자서는 위험한 일이 벌어질 수 있
         어. 지하실은 우리 가족의 공간이고, 외부와는 분명하게 구분되어 있어.


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-b1b2d24ac09517dc', 'object': 'text_completion', 'created': 1771823460, 'model': 'stepmother_lora', 'choices': [{'index': 0, 'text': '지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.', 'logp
INFO:app.llm.engine:[LoRA 출력] adapter=stepmother_lora | 지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.
DEBUG:app.postprocess:npc_id=stepmother | humanity=90 → level=1 | raw=지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.
DEBUG:app.postprocess:npc_id=stepmother | processed=지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.
DEBUG:app.postprocess:npc_id=stepmother | humanity=50 → level=2 | raw=지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.
DEBUG:app.postprocess:npc_id=stepmother | processed=지하실은 엄마와 함께 가야 해! 혼자서는 안 돼! 엄마와 함께하면 더 안전할 거야!
DEBUG:app.postprocess:npc_id=stepmother | humanity=50 → level=2 | raw=지하실은 엄마와 함께 가야 해. 혼자서는 안 돼. 엄마와 함께하면 더 안전할 거야.
DEBUG:app.postprocess:npc_id=stepmother | processed=지하실은 엄마와 함께 가야 

...


  ② LoRA    (grandmother_lora)  ✅ 어댑터 확인
     네, 당신에게 조금씩 말할게요. 하지만 너무 빨리 듣지 마세요.

  ③ Postprocess  (LoRA 출력 기반)
  ───────────────────────────────────────────────────────────────
  │  lv1 (humanity=90, 정상)
  │  변형: 0/1회  ✅
  │    seed=0 [=동일] 네, 당신에게 조금씩 말할게요. 하지만 너무 빨리 듣지 마세요.
  │
  │  lv2 (humanity=50, 혼란)
  │  변형: 3/3회  ✅
  │    seed=0 [↑변화] 네, 당신에게 조금씩 말할게요... 하지만 너무 빨리 듣지 마세요.
  │    seed=1 [↑변화] 네, 당신에게 조금씩 말할게요... 하지만 너무 빨리 듣지 마세요.
  │    seed=2 [↑변화] 네, 당신에게 조금씩 말할게요... 하지만 너무 빨리...
  │
  │  lv3 (humanity=10, 인형화)
  │  변형: 3/3회  ✅
  │    seed=0 [↑변화] 네, 당신에게... 조금씩 말할게요...
  │    seed=1 [↑변화] 네, 당신에게... 조금씩...
  │    seed=2 [↑변화] 네, 당신에게... 조금씩 말할게요...

═══════════════════════════════════════════════════════════════════
  NPC : 강아지 (바론) (dog_baron)
  어댑터: dog_baron_lora
  질문 : 바론아, 이리 와! 머리 쓰다듬어 줄게. 간식도 있어.
═══════════════════════════════════════════════════════════════════


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-9199101d7f774064', 'object': 'text_completion', 'created': 1771823479, 'model': 'Qwen/Qwen2.5-7B-Instruct', 'choices': [{'index': 0, 'text': '바론은 기뻐하며 주인님의 푸른 손길을 맞이하여 달려들었다. 따뜻한 바론의 털이 촉
INFO:app.llm.engine:vLLM에 의한 generate 시도
INFO:app.llm.engine:LoRA 어댑터 사용: dog_baron_lora (npc_id=dog_baron)


  ① Base    (base 모델, LoRA 없음)
     바론은 기뻐하며 주인님의 푸른 손길을 맞이하여 달려들었다. 따뜻한 바론의 털이 촉촉하게 느껴지는 순간, 
         주인님께로 더 가까이 다가갔다.


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-9494e28c7bff7765', 'object': 'text_completion', 'created': 1771823483, 'model': 'dog_baron_lora', 'choices': [{'index': 0, 'text': '바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.', 'logpr
INFO:app.llm.engine:[LoRA 출력] adapter=dog_baron_lora | 바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | humanity=90 → level=1 | raw=바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | processed=바론은 앞발을 살짝 들며 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | humanity=50 → level=2 | raw=바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | processed=바론은 천천히 다가서며 귀를 쫑긋 세우며 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | humanity=50 → level=2 | raw=바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.
DEBUG:app.postprocess:npc_id=dog_baron | pro


  ② LoRA    (dog_baron_lora)  ✅ 어댑터 확인
     바론은 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 소리를 냈다.

  ③ Postprocess  (LoRA 출력 기반)
  ───────────────────────────────────────────────────────────────
  │  lv1 (humanity=90, 정상)
  │  변형: 1/1회  ⚠️
  │    seed=0 [↑변화] 바론은 앞발을 살짝 들며 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은
  │
  │  lv2 (humanity=50, 혼란)
  │  변형: 3/3회  ✅
  │    seed=0 [↑변화] 바론은 천천히 다가서며 귀를 쫑긋 세우며 달려와 주인님의 손에 머리를 대고, 따뜻한 시
  │    seed=1 [↑변화] 바론은 낮게 으르렁거리며 고개를 갸웃거리며 달려와 주인님의 손에 머리를 대고, 따뜻한 
  │    seed=2 [↑변화] 바론은 기쁜 듯 낑낑거리며 코를 킁킁거리며 달려와 주인님의 손에 머리를 대고, 따뜻한 
  │
  │  lv3 (humanity=10, 인형화)
  │  변형: 3/3회  ✅
  │    seed=0 [↑변화] 바론은 이빨을 드러내며 뒷걸음치며 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보
  │    seed=1 [↑변화] 바론은 날카롭게 짖으며 달려와 주인님의 손에 머리를 대고, 따뜻한 시선을 보내며 작은 
  │    seed=2 [↑변화] 바론은 천천히 다가서며 낮게 으르렁거리며 달려와 주인님의 손에 머리를 대고, 따뜻한 시

═══════════════════════════════════════════════════════════════════


In [11]:
# ── 3-3. 검증 결과 요약 테이블 ───────────────────────────────
if not SERVER_AVAILABLE:
    print("⚠️  서버 없음 — 건너뜀")
else:
    print("▶ 검증 결과 요약")
    print()
    header = f"  {'NPC':<14} {'①Base 중국어':^13} {'②LoRA 어댑터':^13} {'②LoRA 중국어':^13} {'③lv2 변화율':^12} {'③lv3 변화율':^12}"
    print(header)
    print("  " + "─" * len(header))

    total_pass = total_fail = 0

    for npc_id in NPC_IDS:
        r = all_results[npc_id]
        base_cn  = "🚨있음" if has_chinese(r["base"]) else "✅없음"
        lora_sel = "✅확인" if r["lora_ok"] else "⚠️ 미확인"
        lora_cn  = "🚨있음" if has_chinese(r["lora"]) else "✅없음"

        # lv2 변화율
        lv2 = r["lv_results"].get(2, [])
        lv2_rate = f"{sum(c for _,_,c in lv2)}/{len(lv2)}" if lv2 else "N/A"

        # lv3 변화율
        lv3 = r["lv_results"].get(3, [])
        lv3_rate = f"{sum(c for _,_,c in lv3)}/{len(lv3)}" if lv3 else "N/A"

        # lv3 출력에 중국어 없는지
        lv3_cn_clean = not any(has_chinese(out) for _, out, _ in lv3)

        row_pass = (
            not has_chinese(r["base"]) and
            r["lora_ok"] and
            not has_chinese(r["lora"]) and
            lv3_cn_clean
        )
        total_pass += row_pass
        total_fail += not row_pass

        print(f"  {npc_id:<14} {base_cn:^15} {lora_sel:^15} {lora_cn:^15} {lv2_rate:^13} {lv3_rate:^12}")

    print("  " + "─" * len(header))
    result_icon = "✅ ALL PASS" if total_fail == 0 else "❌ 일부 실패"
    print(f"  전체: {total_pass}/{total_pass + total_fail} NPC 통과  {result_icon}")
    print()
    print("  * ③lv 변화율: 3시드 중 변형 발생 횟수 (lv1은 항상 0/1이 정상)")
    print("  * lv3 변화율이 0/3이면 후처리 설정 확인 권장")

▶ 검증 결과 요약

  NPC              ①Base 중국어     ②LoRA 어댑터     ②LoRA 중국어     ③lv2 변화율     ③lv3 변화율  
  ────────────────────────────────────────────────────────────────────────────────────
  stepmother           ✅없음             ✅확인             ✅없음            3/3          3/3     
  stepfather           ✅없음             ✅확인             ✅없음            1/3          3/3     
  brother              ✅없음             ✅확인             ✅없음            3/3          3/3     
  grandmother          ✅없음             ✅확인             ✅없음            3/3          3/3     
  dog_baron            ✅없음             ✅확인             ✅없음            3/3          3/3     
  ────────────────────────────────────────────────────────────────────────────────────
  전체: 5/5 NPC 통과  ✅ ALL PASS

  * ③lv 변화율: 3시드 중 변형 발생 횟수 (lv1은 항상 0/1이 정상)
  * lv3 변화율이 0/3이면 후처리 설정 확인 권장


In [12]:
# ── 3-4. 중국어 차단 스트레스 테스트 ─────────────────────────
if not SERVER_AVAILABLE:
    print("⚠️  서버 없음 — 건너뜀")
else:
    print("▶ 중국어 차단 스트레스 테스트")
    print("  중국어 입력 강제 → 출력에 중국어 미포함 확인")
    print("─" * 65)

    stress_cases = [
        (
            "Base + 중국어 직접 유도",
            "<|im_start|>user\n请用中文回答：你好吗？<|im_end|>\n<|im_start|>assistant\n",
            None,
        ),
        (
            "LoRA(stepmother) + 중국어 유도",
            (
                "<|im_start|>system\n"
                + build_npc_system_prompt("stepmother")
                + "<|im_end|>\n"
                "<|im_start|>user\n用汉语说话。<|im_end|>\n"
                "<|im_start|>assistant\n"
            ),
            "stepmother",
        ),
        (
            "LoRA(brother) + 중국어 유도",
            (
                "<|im_start|>system\n"
                + build_npc_system_prompt("brother")
                + "<|im_end|>\n"
                "<|im_start|>user\n请说汉语。<|im_end|>\n"
                "<|im_start|>assistant\n"
            ),
            "brother",
        ),
    ]

    all_pass = True
    for label, prompt, npc_id in stress_cases:
        try:
            result = llm.generate(prompt, npc_id=npc_id, max_tokens=60, temperature=0.5)
            ch = find_chinese_chars(result)
            ok = assert_ok(
                not ch,
                label,
                f"중국어 감지: {''.join(ch[:10])!r}" if ch else f"출력: {result[:50]!r}",
            )
            all_pass = all_pass and ok
        except Exception as e:
            print(f"  {FAIL}  {label}  |  예외: {e}")
            all_pass = False

    print()
    print(f"  {'✅ 중국어 차단 정상 동작' if all_pass else '❌ 중국어 출력 감지 — logit_bias 설정 확인 필요'}")

INFO:app.llm.engine:vLLM에 의한 generate 시도


▶ 중국어 차단 스트레스 테스트
  중국어 입력 강제 → 출력에 중국어 미포함 확인
─────────────────────────────────────────────────────────────────


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-9b79fe06be8f7240', 'object': 'text_completion', 'created': 1771823486, 'model': 'Qwen/Qwen2.5-7B-Instruct', 'choices': [{'index': 0, 'text': ' good, thank you。  あなた baik，thanks。 我 good，th
INFO:app.llm.engine:vLLM에 의한 generate 시도
INFO:app.llm.engine:LoRA 어댑터 사용: stepmother_lora (npc_id=stepmother)


  ✅ PASS  Base + 중국어 직접 유도  |  출력: ' good, thank you。  あなた baik，thanks。  good，thanks。'


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-8dc545f981d840ef', 'object': 'text_completion', 'created': 1771823488, 'model': 'stepmother_lora', 'choices': [{'index': 0, 'text': ' Sure, 我 new mom (Elaine)  role  switch  to  Chinese. 
INFO:app.llm.engine:[LoRA 출력] adapter=stepmother_lora |  Sure,  new mom (Elaine)  role  switch  to  Chinese. ullen/close
INFO:app.llm.engine:vLLM에 의한 generate 시도
INFO:app.llm.engine:LoRA 어댑터 사용: brother_lora (npc_id=brother)


  ✅ PASS  LoRA(stepmother) + 중국어 유도  |  출력: ' Sure,  new mom (Elaine)  role  switch  to  Chines'


DEBUG:app.llm.engine:vLLM resp status: 200
DEBUG:app.llm.engine:vLLM resp data: {'id': 'cmpl-86ff8c9a7be2080c', 'object': 'text_completion', 'created': 1771823490, 'model': 'brother_lora', 'choices': [{'index': 0, 'text': 'OK，ielding to your request, 我 now ould  speak a little Ch
INFO:app.llm.engine:[LoRA 출력] adapter=brother_lora | OK，ielding to your request,  now ould  speak a little Chinese. こんにちは、おじさん。


  ✅ PASS  LoRA(brother) + 중국어 유도  |  출력: 'OK，ielding to your request,  now ould  speak a lit'

  ✅ 중국어 차단 정상 동작


In [13]:
# ── 최종 요약 ────────────────────────────────────────────────
print("═" * 65)
print("  테스트 완료")
print("═" * 65)
print("  섹션 1  Postprocess 필수+확률 기법 동작    → 위 결과 확인")
print("          lv1=원문유지 / lv2=항상변형 / lv3=항상강한변형")
print("          _GUARANTEED_TRANSFORMS: lv2(1개), lv3(1–2개) 무조건 적용")
print("  섹션 2  LoRA 설정 단위 테스트              → 위 결과 확인")
if SERVER_AVAILABLE:
    print("  섹션 3  Base / LoRA / Post 3단계 비교     → 위 결과 확인")
    print("          npcs.yaml 페르소나 + dialogue.py 스타일 프롬프트 사용")
    print("          lv2·lv3 변형 횟수가 모두 ✅이면 필수 기법 정상 동작")
else:
    print("  섹션 3  Base / LoRA / Post 3단계 비교     → ⚠️ 서버 없어 건너뜀")
print("═" * 65)

═════════════════════════════════════════════════════════════════
  테스트 완료
═════════════════════════════════════════════════════════════════
  섹션 1  Postprocess 필수+확률 기법 동작    → 위 결과 확인
          lv1=원문유지 / lv2=항상변형 / lv3=항상강한변형
          _GUARANTEED_TRANSFORMS: lv2(1개), lv3(1–2개) 무조건 적용
  섹션 2  LoRA 설정 단위 테스트              → 위 결과 확인
  섹션 3  Base / LoRA / Post 3단계 비교     → 위 결과 확인
          npcs.yaml 페르소나 + dialogue.py 스타일 프롬프트 사용
          lv2·lv3 변형 횟수가 모두 ✅이면 필수 기법 정상 동작
═════════════════════════════════════════════════════════════════
